In [1]:
from datetime import date, timedelta, datetime
import pandas as pd
import numpy as np

# --- Fechas ---
weeks = 78


In [2]:
import xmlrpc.client


# Conexión con Odoo (manteniendo tus credenciales)
username = "juan.cano@donsson.com"  # tu usuario
password = "1000285668"         # tu contraseña
url = "https://donsson.com"     # URL del servidor
db = "Donsson_produccion" # nombre de la base de datos


# --- Autenticación ---
common = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/common")
uid = common.authenticate(db, username, password, {})
models = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/object")


# --- Fechas automáticas ---
hoy = date.today()
fecha_fin = hoy.strftime("%Y-%m-%d")
fecha_inicio = (hoy - timedelta(weeks=weeks)).strftime("%Y-%m-%d")

# --- 1) Buscar facturas válidas (account.invoice) ---

invoice_domain = [
    ("date_invoice", ">=", fecha_inicio),
    ("date_invoice", "<=", fecha_fin),
    ("type", "=", "out_invoice"),    # solo ventas
    ("state", "in", ["open", "paid"])
]

invoice_ids = models.execute_kw(
    db, uid, password,
    "account.invoice", "search",
    [invoice_domain]
)
print(f"Facturas encontradas: {len(invoice_ids)}")

# --- 2) Descargar las líneas de esas facturas (account.invoice.line) ---

# Campos de la LÍNEA de factura. Eliminamos 'number', 'user_id', 'section_id', 'partner_id' porque irán en la factura.
line_fields = ["product_id", "quantity", "price_subtotal", "invoice_id","create_date","origin"] 

records = []
limit = 20000
offset = 0

while True:
    result = models.execute_kw(
        db, uid, password,
        "account.invoice.line", "search_read",
        [[("invoice_id", "in", invoice_ids)]],
        {"fields": line_fields, "limit": limit, "offset": offset}
    )
    if not result:
        break
    records.extend(result)
    offset += limit
    print(f"Descargados {len(records)} registros de líneas...")

# --- 3) Pasar a DataFrame de líneas ---
line_df = pd.DataFrame(records).fillna(0)


# Separar product_id
line_df["product_id_num"] = line_df["product_id"].apply(
    lambda x: x[0] if isinstance(x, (list, tuple)) else None
)
line_df["product_name"] = line_df["product_id"].apply(
    lambda x: x[1] if isinstance(x, (list, tuple)) else str(x)
)

# Separar invoice_id
line_df["invoice_id_num"] = line_df["invoice_id"].apply(
    lambda x: x[0] if isinstance(x, (list, tuple)) else None
)
line_df["invoice_name"] = line_df["invoice_id"].apply(
    lambda x: x[1] if isinstance(x, (list, tuple)) else str(x)
)

# Convertir fecha a datetime
line_df["date_invoice"] = pd.to_datetime(line_df["create_date"], errors="coerce")

# Eliminar las columnas originales problemáticas
line_df = line_df.drop(columns=["invoice_id","create_date"])

print(f"Total de líneas descargadas: {len(line_df)}")

# ----------------------------------------------------
# --- 4) Descargar los campos adicionales de Factura (account.invoice) ---
# ----------------------------------------------------
# Añadimos los campos que quieres: number, user_id, section_id, y también partner_id y store_id
invoice_fields = ["id", "store_id", "number", "user_id", "section_id", "partner_id"]
invoices = models.execute_kw(
    db, uid, password,
    "account.invoice", "read",
    [invoice_ids], # Solo las facturas que encontramos
    {"fields": invoice_fields}
)
invoice_df = pd.DataFrame(invoices)

# --- 5) Procesar campos de la factura ---

# Separar store_id
invoice_df["store_name"] = invoice_df["store_id"].apply(
    lambda x: x[1] if isinstance(x, (list, tuple)) else str(x)
)

# Separar user_id (Vendedor)
invoice_df["salesperson_name"] = invoice_df["user_id"].apply(
    lambda x: x[1] if isinstance(x, (list, tuple)) else None
)

# Separar section_id (Equipo de Ventas)
invoice_df["sales_team_name"] = invoice_df["section_id"].apply(
    lambda x: x[1] if isinstance(x, (list, tuple)) else None
)

# Separar partner_id (Cliente/Partner)
invoice_df["partner_id_num"] = invoice_df["partner_id"].apply(
    lambda x: x[0] if isinstance(x, (list, tuple)) else None
)
# El nombre del partner es el segundo elemento de la tupla (si existe)
invoice_df["client_name_inv"] = invoice_df["partner_id"].apply(
    lambda x: x[1] if isinstance(x, (list, tuple)) else None
)


# Eliminar columnas originales no deseadas o ya procesadas
invoice_df = invoice_df.drop(columns=["store_id", "user_id", "section_id", "partner_id"])


# ----------------------------------------------------
# --- 6) Fusionar DataFrames ---
# ----------------------------------------------------

# Fusionamos las líneas de factura (line_df) con los datos de las facturas (invoice_df)
df_merged = line_df.merge(
    invoice_df, 
    left_on="invoice_id_num", 
    right_on="id", 
    how="left"
)

# Limpieza final de columnas de IDs de factura
df_merged = df_merged.drop(columns=["invoice_id_num", "product_id_num"])


df_merged['origin'] = df_merged['origin'].astype('string')
df_merged["product_id"] = df_merged["product_id"].astype(str)

# Guardar
df_merged.to_parquet(f"ventashistoricas{weeks}semanas.parquet", index=False)

print(f"Total de líneas descargadas con información de factura: {len(df_merged)}")

#para 52 semanas se demora aproximadamente 6.5 minutos
#para 56 semanas se demora aproximadamente 7 minutos
#para 105 semanas se demora aproximadamente 14 minutos

Facturas encontradas: 75398
Descargados 20000 registros de líneas...
Descargados 40000 registros de líneas...
Descargados 60000 registros de líneas...
Descargados 80000 registros de líneas...
Descargados 100000 registros de líneas...
Descargados 120000 registros de líneas...
Descargados 140000 registros de líneas...
Descargados 160000 registros de líneas...
Descargados 180000 registros de líneas...
Descargados 200000 registros de líneas...
Descargados 220000 registros de líneas...
Descargados 240000 registros de líneas...
Descargados 260000 registros de líneas...
Descargados 280000 registros de líneas...
Descargados 290315 registros de líneas...
Total de líneas descargadas: 290315
Total de líneas descargadas con información de factura: 290315


In [17]:
df = df_merged.copy()

In [18]:
import re
import unicodedata



mapa_sucursales = {
    'BD1': 'PRINCIPAL COTA',
    'BD2': 'SUCURSAL CALLE 6',
    'BD3': 'SUCURSAL VALLADOLID',
    'BD4': 'SUCURSAL NORTE',
    'BD5': 'SUCURSAL MEDELLIN',
    'BD6': 'SUCURSAL BARRANQUILLA',
    'BD7':' SUCURSAL BUCARAGAMNGA ',
    'BD8':'MOSTRADOR COTA',
    'BD9':'SUCURSAL CALI',
    'BD11': 'CUMMINS DE LOS ANDES',
    'BD12': 'CUMMINS DE LOS ANDES MEDELLIN',
    'BD13': 'CSS CONSTRUCTORES',
    'BD14': 'ANTIOQUEÑA DE LUBRICANTES SGP SAS',
    'BD15': 'ASEO CAPITAL',
    'BD16': 'COVIANDES',
    'BD18': 'CEMEX BOSA',
    'BD19': 'Cemex ibague (eliminar)',
    'BD21': 'CEMEX CENTENARIO',
    'BD22': 'PENDIENTE POR FACTURAR CIERRE CONT.',
    'BD24': 'FILTRO EN REPROCESO',
    'BD26': 'PRESTAMOS INTERNOS',
    'BD31': 'EXP.CTP',
    'BD32': 'EXP. ECUADOR',
    'BD34':'EXP.DONSSON.USA',
    'BD40': 'DESCUADRE B1',
    'BD MP':'MATERIA PRIMA',
    'BD41':'BODEGA VENDEDOR WILMER GIL',
    'BD33':'EXP.REP.DOMINICANA',
    'BD42':'STOCK SIGMA ENERGY'
    # Agrega más códigos si los encuentras en tus datos, siguiendo este formato.
    }




def clasificar_sucursal(invoice_name):
    if pd.isna(invoice_name):
        return "VENDEDOR EXTERNO"
    if "Mostrador" in invoice_name:
        return "MOSTRADOR"
    return "VENDEDOR EXTERNO"
    
df["Sucursal_tipo"] = df["invoice_name"].apply(clasificar_sucursal)


df["client_name"] = df["client_name_inv"].str.split("] ", expand=True)[1].str.strip()



#Quitar las facturas que salen como SO son no vendibles
df = df[~(df["origin"].str.contains("SO"))] 


# Normalizar los separadores para que todos sean iguales
df["origin_norm"] = df["origin"].str.replace(r"[\\/]", "/", regex=True)

# Extraer solo el BD y el número (ejemplo: BD11, BD9, etc.)
df["origen"] = df["origin_norm"].str.extract(r"^(BD\d+)")

df["Bodega"] = df["origen"].map(mapa_sucursales)
df["Bodega"] = df["Bodega"].fillna(df["store_name"])

df = df.drop(columns=["origin_norm"])

df = df.drop(columns=["origin"])

df["product_ref"] = df["product_name"].str.extract(r"\[([A-Z0-9]+)\]")

df["product_nom"] = df["product_name"].str.extract(r"\]\s*([A-Z]\w+)")


df["date_invoice"] = pd.to_datetime(df["date_invoice"])
df["mes"] = df["date_invoice"].dt.month
df["año"] = df["date_invoice"].dt.year
df["dia"] = df["date_invoice"].dt.day

df = df[~(df["product_ref"].isna())]

pd.set_option('display.max_columns', None)

df['marca'] = df['product_ref'].apply(
    lambda x: 'BALDWIN' if str(x).endswith('125') else
              'DONSSON' if str(x).endswith('025') else
              'AUT*PARTS' if str(x).endswith('189') else
              'RACOR BRASIL' if str(x).endswith('137') else
              'RACOR USA' if str(x).endswith('138') else
              'OTRA'
)

In [19]:
df.head()

,product_id,price_subtotal,id_x,quantity,product_name,invoice_name,date_invoice,number,id_y,store_name,salesperson_name,sales_team_name,partner_id_num,client_name_inv,Sucursal_tipo,client_name,origen,Bodega,product_ref,product_nom,mes,año,dia,marca
0,"[19616, '[BLS00220125] GS220 FILTRO ACEITE HID...",186000.0,1647778,1.0,[BLS00220125] GS220 FILTRO ACEITE HIDRAULICO C...,PV6E20013 Mostrador Barranquilla T3/3738,2025-11-04 14:23:51,PV6E20013,446645,SUCURSAL BARRANQUILLA,NEYDER CRESPO YEPES,SUCURSALES / BARRANQUILLA / Ventas mostrador B...,34633,[CO9012522615] LG MANTENIMIENTO PREVENTIVO S.A.S,MOSTRADOR,LG MANTENIMIENTO PREVENTIVO S.A.S,<NA>,SUCURSAL BARRANQUILLA,BLS00220125,GS220,11,2025,4,BALDWIN
1,"[47503, '[BAE09107125] DA9207 FILTRO AIRE SEGU...",91200.0,1647775,1.0,[BAE09107125] DA9207 FILTRO AIRE SEGURIDAD- BO...,PV1E11523 Mostrador Cota T2/4593,2025-11-04 14:20:09,PV1E11523,446644,PRINCIPAL COTA,YURI NATALIA GAONA,SUCURSALES / Ventas Mostrador Cota,32042,[CO9003605207] PARMER SAS,MOSTRADOR,PARMER SAS,<NA>,PRINCIPAL COTA,BAE09107125,DA9207,11,2025,4,BALDWIN
2,"[19019, '[BCS30168125] GS168A1 FILTRO COMBUSTI...",71200.0,1647777,1.0,[BCS30168125] GS168A1 FILTRO COMBUSTIBLE SEPAR...,PV1E11523 Mostrador Cota T2/4593,2025-11-04 14:20:09,PV1E11523,446644,PRINCIPAL COTA,YURI NATALIA GAONA,SUCURSALES / Ventas Mostrador Cota,32042,[CO9003605207] PARMER SAS,MOSTRADOR,PARMER SAS,<NA>,PRINCIPAL COTA,BCS30168125,GS168A1,11,2025,4,BALDWIN
3,"[47502, '[BAE08207125] DA8207 FILTRO AIRE - BO...",136800.0,1647774,1.0,[BAE08207125] DA8207 FILTRO AIRE - BOBCAT (125...,PV1E11523 Mostrador Cota T2/4593,2025-11-04 14:20:09,PV1E11523,446644,PRINCIPAL COTA,YURI NATALIA GAONA,SUCURSALES / Ventas Mostrador Cota,32042,[CO9003605207] PARMER SAS,MOSTRADOR,PARMER SAS,<NA>,PRINCIPAL COTA,BAE08207125,DA8207,11,2025,4,BALDWIN
4,"[19560, '[BLS00022125] GS022 FILTRO ACEITE CAT...",30800.0,1647776,1.0,"[BLS00022125] GS022 FILTRO ACEITE CATERPILLAR,...",PV1E11523 Mostrador Cota T2/4593,2025-11-04 14:20:09,PV1E11523,446644,PRINCIPAL COTA,YURI NATALIA GAONA,SUCURSALES / Ventas Mostrador Cota,32042,[CO9003605207] PARMER SAS,MOSTRADOR,PARMER SAS,<NA>,PRINCIPAL COTA,BLS00022125,GS022,11,2025,4,BALDWIN


## VENTAS PERDIDAS

In [20]:
import xmlrpc.client
from datetime import date, timedelta
import pandas as pd

# ===============================
# 1. Conexión con Odoo
# ===============================

ODOO_URL = "https://donsson.com"
ODOO_DB = "Donsson_produccion"
ODOO_USERNAME = "juan.cano@donsson.com"
ODOO_PASSWORD = "1000285668"

# Autenticación
common = xmlrpc.client.ServerProxy(f"{ODOO_URL}/xmlrpc/2/common")
uid = common.authenticate(ODOO_DB, ODOO_USERNAME, ODOO_PASSWORD, {})
if not uid:
    print("Error de autenticación. Verifica tus credenciales.")
    exit()
models = xmlrpc.client.ServerProxy(f"{ODOO_URL}/xmlrpc/2/object")

# ===============================
# 2. Definir rango de semanas
# ===============================

# Número de semanas que quieres descargar (puedes cambiarlo)
NUM_SEMANAS = weeks

# Fecha de hoy
hoy = date.today()
# Fecha de inicio según número de semanas atrás
fecha_inicio = hoy - timedelta(weeks=NUM_SEMANAS)

FECHA_INICIO = fecha_inicio.strftime("%Y-%m-%d")
FECHA_FIN = hoy.strftime("%Y-%m-%d")

print(f"📅 Descargando registros desde {FECHA_INICIO} hasta {FECHA_FIN}")

# ===============================
# 3. Leer ventas perdidas
# ===============================

lost_sales_ids = models.execute_kw(
    ODOO_DB, uid, ODOO_PASSWORD,
    "ventas.perdidas", "search",
    [[["fecha", ">=", FECHA_INICIO], ["fecha", "<=", FECHA_FIN], ["almacenamiento_tipo", "=", "agotado"]]]
)

lost_sales_data = models.execute_kw(
    ODOO_DB, uid, ODOO_PASSWORD,
    "ventas.perdidas", "read",
    [lost_sales_ids],
    {"fields": ["origen", "store_id", "cliente_id", "fecha", "product_ref",
                "cantidad", "cantidad_existencia", "cantidad_reservada",
                "almacenamiento_tipo"]}
)

df_vp = pd.DataFrame(lost_sales_data)

# ===============================
# 4. Normalizar columnas
# ===============================

for col in ["store_id", "cliente_id"]:
    df_vp[col.replace('_id', '_name')] = df_vp[col].apply(
        lambda x: x[1] if isinstance(x, (list, tuple)) and len(x) > 1 else str(x) if x else None
    )
    df_vp[col + "_num"] = df_vp[col].apply(
        lambda x: x[0] if isinstance(x, (list, tuple)) and len(x) > 0 else None
    )

# Eliminar columnas originales
df_vp = df_vp.drop(columns=["id", "store_id", "cliente_id"], errors='ignore')

# ===============================
# 4.1. Forzar tipos de columnas antes de exportar
# ===============================
for col in df_vp.columns:
    if col.endswith("_num"):
        df_vp[col] = pd.to_numeric(df_vp[col], errors="coerce").astype("Int64")
    else:
        df_vp[col] = df_vp[col].astype("string")

df_vp = df_vp.copy()



📅 Descargando registros desde 2024-05-07 hasta 2025-11-04


In [21]:
df_vp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97736 entries, 0 to 97735
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   cantidad_reservada   97736 non-null  string
 1   product_ref          97736 non-null  string
 2   cantidad             97736 non-null  string
 3   almacenamiento_tipo  97736 non-null  string
 4   fecha                97736 non-null  string
 5   cantidad_existencia  97736 non-null  string
 6   origen               97736 non-null  string
 7   store_name           97736 non-null  string
 8   store_id_num         97736 non-null  Int64 
 9   cliente_name         97736 non-null  string
 10  cliente_id_num       97736 non-null  Int64 
dtypes: Int64(2), string(9)
memory usage: 8.4 MB


In [22]:
# ===============================
# Filtrar almacenamiento agotado
# ===============================
df_vp = df_vp[df_vp["almacenamiento_tipo"].str.lower() == "agotado"]

# ===============================
# Asegurar tipos correctos
# ===============================
df_vp = df_vp.copy()
df_vp["fecha"] = pd.to_datetime(df_vp["fecha"], errors="coerce")

# Numéricos
for col in ["cantidad", "cantidad_existencia", "cantidad_reservada"]:
    df_vp[col] = pd.to_numeric(df_vp[col], errors="coerce").fillna(0).clip(lower=0)

# ===============================
# Reglas Odoo vectorizadas
# ===============================
is_cot = df_vp["origen"].fillna("").str.lower() == "cotizacion"
ignore_mask = df_vp["cantidad"] >= 100

ajuste = np.where(
    is_cot,
    df_vp["cantidad"] - df_vp["cantidad_existencia"] - df_vp["cantidad_reservada"],
    df_vp["cantidad"] - df_vp["cantidad_reservada"]
)

# Aplicar reglas de descarte y piso en cero
ajuste = np.where(ignore_mask, 0, ajuste)
ajuste = np.where(ajuste > 0, ajuste, 0)

df_vp["ventas_perdidas"] = ajuste.astype(float)

# ===============================
# Columnas temporales
# ===============================
df_vp["Semana"] = df_vp["fecha"].dt.to_period("M").dt.start_time
df_vp["ano"]   = df_vp["Semana"].dt.year
df_vp["mes"]   = df_vp["Semana"].dt.month
df_vp["dia"]   = df_vp["Semana"].dt.day

# ===============================
# Filtro adicional: excluir SERV y CARCASA
# ===============================
mask_excluir = ~df_vp["product_ref"].str.contains("SERV|CARCASA", case=False, na=False)
df_vp = df_vp[mask_excluir]

# ===============================
# Agrupación por tienda + producto + semana
# ===============================
lost_by_week = (
    df_vp.groupby(["store_name", "product_ref", "mes", "ano", "Semana", "dia"])
    .agg(
        lost_sales=("ventas_perdidas", "sum"),   # suma total de ventas perdidas
        veces_vp=("ventas_perdidas", "count")    # número de veces que hubo pérdida
    )
    .reset_index()
)

# Mostrar resultado agrupado
vp_month = lost_by_week


In [23]:
vp_month[vp_month["product_ref"]=="DAB08177025"]

,store_name,product_ref,mes,ano,Semana,dia,lost_sales,veces_vp
5302,PRINCIPAL COTA,DAB08177025,5,2024,2024-05-01,1,1.0,1
5303,PRINCIPAL COTA,DAB08177025,6,2025,2025-06-01,1,1.0,1
5304,PRINCIPAL COTA,DAB08177025,7,2025,2025-07-01,1,6.0,5
5305,PRINCIPAL COTA,DAB08177025,8,2025,2025-08-01,1,7.0,3
5306,PRINCIPAL COTA,DAB08177025,9,2025,2025-09-01,1,4.0,4
5307,PRINCIPAL COTA,DAB08177025,10,2025,2025-10-01,1,4.0,3
16142,SUCURSAL BARRANQUILLA,DAB08177025,9,2025,2025-09-01,1,6.0,1
16143,SUCURSAL BARRANQUILLA,DAB08177025,10,2024,2024-10-01,1,0.0,1
16144,SUCURSAL BARRANQUILLA,DAB08177025,10,2025,2025-10-01,1,1.0,1
24577,SUCURSAL BUCARAMANGA,DAB08177025,2,2025,2025-02-01,1,0.0,1


In [25]:
from datetime import datetime

# Fecha y hora actual en formato YYYYMMDD_HHMMSS
timestamp = datetime.now().strftime("%Y%m%d")

# Nombre dinámico
filename = f"/home/donsson/proyectos/VENTAS/prom_ventas/promedio_ventas_{timestamp}.xlsx"

# Guardar el Excel
df.to_excel(filename, index=False)

print(f"Archivo guardado: {filename}")


df_vp.to_excel("/home/donsson/proyectos/VENTAS/prom_ventas/vp_mes.xlsx")

#Tiempo promedio 3 minutos

Archivo guardado: /home/donsson/proyectos/VENTAS/prom_ventas/promedio_ventas_20251104.xlsx


## VISUM

In [11]:
clientes = {"VISUM LTDA",
"VILMAR LUBRICANTES S.A.S.",
"ESTACION Y LUBRICANTES SAN ANTONIO GNV S.A.S.", 
"SODATRANS LTDA",
"ESTACION Y LUBRICANTES CASABLANCA S.A.S"}

fecha_inicial = "09/1/2024"
fecha_final = "09/30/2025"

df_h = df[(df["client_name"].isin(clientes)) & (df["date_invoice"] >= fecha_inicial) & (df["date_invoice"] <= fecha_final) & (df["salesperson_name"] == "YEIS OSWALDO MUÑOZ GUERRERO")  ]

df_h = df_h.drop(columns=["product_id","id_x","id_y","partner_id_num","client_name_inv","invoice_name","origen","number"])

In [12]:
df_h["date_invoice"].max()

Timestamp('2025-09-19 14:59:01')

In [13]:
df_h.sample(10)

,price_subtotal,quantity,product_name,date_invoice,store_name,salesperson_name,sales_team_name,Sucursal_tipo,client_name,Bodega,product_ref,product_nom,mes,año,dia,marca
157543,1257600.0,24.0,[BCS00242125] GS242 FILTRO COMBUSTIBLE SEPARAD...,2025-01-30 15:29:12,PRINCIPAL COTA,YEIS OSWALDO MUÑOZ GUERRERO,SUCURSALES / Ventas Credito Norte,VENDEDOR EXTERNO,VISUM LTDA,PRINCIPAL COTA,BCS00242125,GS242,1,2025,30,BALDWIN
129068,830400.0,12.0,[DAB08191025] DA8191 FILTRO AIRE FOTON AUMARK ...,2025-03-19 13:39:25,PRINCIPAL COTA,YEIS OSWALDO MUÑOZ GUERRERO,SUCURSALES / Ventas Credito Norte,VENDEDOR EXTERNO,VISUM LTDA,PRINCIPAL COTA,DAB08191025,DA8191,3,2025,19,DONSSON
157482,251200.0,1.0,[BAP02020125] DA2020 FILTRO AIRE- CAT 416D (12...,2025-01-30 15:29:12,PRINCIPAL COTA,YEIS OSWALDO MUÑOZ GUERRERO,SUCURSALES / Ventas Credito Norte,VENDEDOR EXTERNO,VISUM LTDA,PRINCIPAL COTA,BAP02020125,DA2020,1,2025,30,BALDWIN
131603,85800.0,1.0,[BLS00202125] GS202 FILTRO ACEITE FILTROFILTRO...,2025-03-14 14:42:37,PRINCIPAL COTA,YEIS OSWALDO MUÑOZ GUERRERO,SUCURSALES / Ventas Credito Norte,VENDEDOR EXTERNO,VISUM LTDA,PRINCIPAL COTA,BLS00202125,GS202,3,2025,14,BALDWIN
131595,120800.0,2.0,[BLS00311125] GS311 FILTRO ACEITE BALDWIN - CU...,2025-03-14 14:42:37,PRINCIPAL COTA,YEIS OSWALDO MUÑOZ GUERRERO,SUCURSALES / Ventas Credito Norte,VENDEDOR EXTERNO,VISUM LTDA,PRINCIPAL COTA,BLS00311125,GS311,3,2025,14,BALDWIN
226286,228600.0,6.0,[DAR02594025] DA2594 FILTRO AIRE CAMION KIA 35...,2024-09-17 15:12:42,PRINCIPAL COTA,YEIS OSWALDO MUÑOZ GUERRERO,SUCURSALES / Ventas Credito Norte,VENDEDOR EXTERNO,VISUM LTDA,PRINCIPAL COTA,DAR02594025,DA2594,9,2024,17,DONSSON
108466,104800.0,2.0,"[BHS00435125] GS435 FILTRO HIDRAULICO FORD, NE...",2025-04-25 20:03:23,PRINCIPAL COTA,YEIS OSWALDO MUÑOZ GUERRERO,SUCURSALES / Ventas Credito Norte,VENDEDOR EXTERNO,ESTACION Y LUBRICANTES SAN ANTONIO GNV S.A.S.,PRINCIPAL COTA,BHS00435125,GS435,4,2025,25,BALDWIN
226154,434400.0,24.0,[BLS10050125] GS050 FILTRO ACEITE BALDWIN - FO...,2024-09-17 15:25:41,PRINCIPAL COTA,YEIS OSWALDO MUÑOZ GUERRERO,SUCURSALES / Ventas Credito Norte,VENDEDOR EXTERNO,VISUM LTDA,PRINCIPAL COTA,BLS10050125,GS050,9,2024,17,BALDWIN
206463,75600.0,2.0,[BCS00161125] GS161 FILTRO COMBUSTIBLE IHC (12...,2024-10-24 16:02:08,PRINCIPAL COTA,YEIS OSWALDO MUÑOZ GUERRERO,SUCURSALES / Ventas Credito Norte,VENDEDOR EXTERNO,VILMAR LUBRICANTES S.A.S.,PRINCIPAL COTA,BCS00161125,GS161,10,2024,24,BALDWIN
129037,127200.0,6.0,"[BLS00046125] GS046 FILTRO ACEITE FORD, TOYOTA...",2025-03-19 13:45:43,PRINCIPAL COTA,YEIS OSWALDO MUÑOZ GUERRERO,SUCURSALES / Ventas Credito Norte,VENDEDOR EXTERNO,VISUM LTDA,PRINCIPAL COTA,BLS00046125,GS046,3,2025,19,BALDWIN


In [14]:
df_h[df_h["client_name"]=="VILMAR LUBRICANTES S.A.S."]

,price_subtotal,quantity,product_name,date_invoice,store_name,salesperson_name,sales_team_name,Sucursal_tipo,client_name,Bodega,product_ref,product_nom,mes,año,dia,marca
43839,174800.0,2.0,[BLS00225125] GS225 FILTRO ACEITE TOYOTA MONTA...,2025-08-21 13:50:48,PRINCIPAL COTA,YEIS OSWALDO MUÑOZ GUERRERO,SUCURSALES / Ventas Credito Norte,VENDEDOR EXTERNO,VILMAR LUBRICANTES S.A.S.,PRINCIPAL COTA,BLS00225125,GS225,8,2025,21,BALDWIN
43840,147600.0,3.0,"[BLS00037125] GS037 FILTRO ACEITE MACK, CATERP...",2025-08-21 13:50:48,PRINCIPAL COTA,YEIS OSWALDO MUÑOZ GUERRERO,SUCURSALES / Ventas Credito Norte,VENDEDOR EXTERNO,VILMAR LUBRICANTES S.A.S.,PRINCIPAL COTA,BLS00037125,GS037,8,2025,21,BALDWIN
43841,27400.0,1.0,[BLS00049125] GS049 FILTRO ACEITE JOHN DEERE. ...,2025-08-21 13:50:48,PRINCIPAL COTA,YEIS OSWALDO MUÑOZ GUERRERO,SUCURSALES / Ventas Credito Norte,VENDEDOR EXTERNO,VILMAR LUBRICANTES S.A.S.,PRINCIPAL COTA,BLS00049125,GS049,8,2025,21,BALDWIN
43842,378000.0,6.0,[BLS00311125] GS311 FILTRO ACEITE BALDWIN - CU...,2025-08-21 13:50:48,PRINCIPAL COTA,YEIS OSWALDO MUÑOZ GUERRERO,SUCURSALES / Ventas Credito Norte,VENDEDOR EXTERNO,VILMAR LUBRICANTES S.A.S.,PRINCIPAL COTA,BLS00311125,GS311,8,2025,21,BALDWIN
43843,184800.0,3.0,[BLS00106125] GS106 FILTRO ACEITE FULL-FLOW MA...,2025-08-21 13:50:48,PRINCIPAL COTA,YEIS OSWALDO MUÑOZ GUERRERO,SUCURSALES / Ventas Credito Norte,VENDEDOR EXTERNO,VILMAR LUBRICANTES S.A.S.,PRINCIPAL COTA,BLS00106125,GS106,8,2025,21,BALDWIN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
222768,134000.0,2.0,[DAR03370025] DA3370 FILTRO AIRE - CASE - JCB ...,2024-09-23 17:11:52,PRINCIPAL COTA,YEIS OSWALDO MUÑOZ GUERRERO,SUCURSALES / Ventas Credito Norte,VENDEDOR EXTERNO,VILMAR LUBRICANTES S.A.S.,PRINCIPAL COTA,DAR03370025,DA3370,9,2024,23,DONSSON
222769,136800.0,6.0,[DLS00295189] GS295 FILTRO ACEITE DUAL - CHEVR...,2024-09-23 17:11:52,PRINCIPAL COTA,YEIS OSWALDO MUÑOZ GUERRERO,SUCURSALES / Ventas Credito Norte,VENDEDOR EXTERNO,VILMAR LUBRICANTES S.A.S.,PRINCIPAL COTA,DLS00295189,GS295,9,2024,23,AUT*PARTS
222770,266400.0,6.0,[DAE04304025] DA4304 FILTRO AIRE 2_ CHEVROLET ...,2024-09-23 17:11:52,PRINCIPAL COTA,YEIS OSWALDO MUÑOZ GUERRERO,SUCURSALES / Ventas Credito Norte,VENDEDOR EXTERNO,VILMAR LUBRICANTES S.A.S.,PRINCIPAL COTA,DAE04304025,DA4304,9,2024,23,DONSSON
222771,186300.0,3.0,"[DAR08049025] DA8049 FILTRO AIRE HYUNDAI HD78,...",2024-09-23 17:11:52,PRINCIPAL COTA,YEIS OSWALDO MUÑOZ GUERRERO,SUCURSALES / Ventas Credito Norte,VENDEDOR EXTERNO,VILMAR LUBRICANTES S.A.S.,PRINCIPAL COTA,DAR08049025,DA8049,9,2024,23,DONSSON


In [15]:
#df_h.to_excel("harold_vendedor.xlsx")